## Quadtree Image en Noir et Blanc

In [1]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from sklearn import datasets
import graphviz
import cv2 as cv

## Image en niveau de gris

In [2]:
img_gris=cv.imread("C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot.png",0)
cv.imshow("Image Par defaut",img_gris)
k=cv.waitKey(0)

## Image en couleur

In [3]:
img_color=cv.imread("C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot.png",1)

## Image en noir et blanc

In [4]:
"""img=cv.imread("C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot.png",0)
h,l=img.shape
for i in range(h):
    for j in range (l):
        if img[i][j]>128:
            img[i][j]=255
        else:
            img[i][j]=0
cv.imshow("Image Par defaut",img)
k=cv.waitKey(0)
"""

'img=cv.imread("C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot.png",0)\nh,l=img.shape\nfor i in range(h):\n    for j in range (l):\n        if img[i][j]>128:\n            img[i][j]=255\n        else:\n            img[i][j]=0\ncv.imshow("Image Par defaut",img)\nk=cv.waitKey(0)\n'

In [5]:
_, thres=cv.threshold(img_gris,127,255,cv.THRESH_BINARY)
cv.imshow("Foot",img_gris)
cv.imshow("Test_Seuil",thres)
cv.waitKey(0)
cv.destroyAllWindows()

## Partition de l'image en 4

In [6]:
def split_image(img):
    h,l=img.shape
    if (h>1 and l>1):
        img_0=img[:h//2,:l//2]
        img_1=img[:h//2,l//2:]
        img_2=img[h//2:,:l//2]
        img_3=img[h//2:,l//2:]
        return True,img_0,img_1,img_2,img_3
    else:
        return False,None,None,None,None

In [7]:
CST_BLANC=255
CST_NOIR=0
def Homogeneite(img):
    moyenne=np.mean(img)
    return (moyenne==CST_BLANC or moyenne==CST_NOIR),moyenne

In [8]:
class Node:
    num=0
    def __init__(self,x0,y0,x1,y1):
        self.x0=x0
        self.x1=x1
        self.y0=y0
        self.y1=y1
        self.is_leaf= False
        self.color = None
        self.subnode = [None,None,None,None]
        self.name = self.getName()
    
    def getName(self):
        Node.num += 1
        return "<" + str(Node.num) + ">"
    
    def contains(self,x,y):
        return (x>=self.x0 and x<=self.x1) and (y>=self.y0 and y<=self.y1)
    
    def get_taille(self):
        return (self.x1-self.x0),(self.y1-self.y0)
    
    def display(self,d):  # utilisation de la libraire graphviz pour avoir un joli affichage
            
        info = ";".join([str(self.x0),str(self.y0),str(self.x1),str(self.y1)])
        if self.is_leaf:
            d.node(self.name,"["+str(self.color)+"]"+"\n" + info + "\n")
        else:
            d.node(self.name,"X\n" + info + "\n", color="red")
            for s in self.subnode:
                if s:
                    s.display(d)
                    d.edge(self.name,s.name,s.name)

In [9]:
class DecisionTree:
    maxProf=0
    def __init__(self,img,max_split=30):
        self.split=0
        self.max_split=max_split
        self.tree=None
        self.img=img
        self.nbiter=0
        Node.num=0
        
    def segmentation_recursif(self,img,x=0,y=0):
        """Pas appelée directement"""
        
        node=None
        
        if self.split<self.max_split:
            h,l=img.shape
            node = Node(x,y,x+l,y+h)
        
            node.is_leaf,node.color  = Homogeneite(img) # On appelle la fonction défini plus pour faire notre split
            
            if not node.is_leaf:
                valide,img_a,img_b,img_c,img_d=split_image(img)
                if valide:
                    self.split+=1  
                    if self.split > DecisionTree.maxProf:
                        DecisionTree.maxProf = self.split
                    node.subnode[0]=self.segmentation_recursif(img_a,x,y)
                    node.subnode[1]=self.segmentation_recursif(img_b,x+l//2,y)
                    node.subnode[2]=self.segmentation_recursif(img_c,x,y+h//2)
                    node.subnode[3]=self.segmentation_recursif(img_d,x+l//2,y+h//2)
                    self.split-=1              
        
        return node
    
    def segmentation(self):
        
        self.tree=self.segmentation_recursif(self.img) 
        
        return self.tree
    
    def getleaf(self,x,y,node):
        self.nbiter+=1
        if node:
            if node.is_leaf:
                return node
            else:
                for s in node.subnode:
                    if s.contains(x,y):
                        return self.getleaf(x,y,s)

        return node

    
    def predict(self,x,y):
        self.nbiter=0
        return self.getleaf(x,y,self.tree)
    
    def displayF(self,x,y):
        p=self.predict(x,y)
        if p:
            print("Node:" + str(p.name) + ",recur=" + str(self.nbiter) + " val["+str(x)+","+str(y)+"] = "+str(p.color)+"   rectangle= ("+ str(p.x0)+" ; "+str(p.y0)+" );( "+str(p.x1)+" ; "+str(p.y1)+" ) ")
        else:
            print("Hors zone")
            
            
    def reconstruction_node(self,img,node,taillemin):
        if node:
            if not node.is_leaf:
                tx,ty=node.get_taille()
                if (tx>taillemin) and (ty>taillemin):
                    cv.rectangle(img,(node.x0,node.y0),(node.x1,node.y1),(0,0,255),1)
                    for s in node.subnode:
                        self.reconstruction_node(img,s,taillemin)
                    
    def reconstruction_rec(self,img,taillemin=30):
        reconstruction=img.copy()
        self.reconstruction_node(reconstruction,self.tree,taillemin)
        return reconstruction
    
    def sauvegarde_image_rec(self,img,label,filename,taillemin=10):
        nouvelle_img=self.reconstruction_rec(img,taillemin)
        cv.imshow(label,nouvelle_img)
        cv.imwrite(filename,nouvelle_img)
        k=cv.waitKey(0)
        cv.destroyAllWindows()
        

## Création de l'arbre

In [10]:
Arbre=DecisionTree(thres,max_split=100)
Racine=Arbre.segmentation()
print("Profondeur max:" + str(DecisionTree.maxProf))

Profondeur max:8


## Affichage de l'arbre avec graphviz

In [11]:
d = graphviz.Digraph(format='png')
Racine.display(d)
d.render()

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.356853 to fit


'Digraph.gv.png'

## Recherche d'un point dans l'arbre

In [12]:
Arbre.displayF(12,55)
Arbre.displayF(102,55)
Arbre.displayF(12,515)

Node:<6>,recur=4 val[12,55] = 255.0   rectangle= (0 ; 45 );( 34 ; 68 ) 
Node:<95>,recur=4 val[102,55] = 255.0   rectangle= (68 ; 45 );( 102 ; 68 ) 
Node:<1>,recur=1 val[12,515] = 245.9350223546945   rectangle= (0 ; 0 );( 275 ; 183 ) 


## Reconstruction de l'image

In [13]:
nouvelle_img=Arbre.reconstruction_rec(img_gris,taillemin=20)
cv.imshow("Exemple",nouvelle_img)
k=cv.waitKey(0)
cv.destroyAllWindows()

## Sauvegarde de l'image (reconstruction + affichage + sauvegarde)

In [14]:
Arbre.sauvegarde_image_rec(img_gris,"Foot_gris_20","C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot_gris_20.png",taillemin=20)
Arbre.sauvegarde_image_rec(img_color,"Foot_color_20","C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot_color_20.png",taillemin=20)
Arbre.sauvegarde_image_rec(img_color,"Foot_color_4","C:/Users/pypar/Documents/M1 MMAA/Semestre 8/ProjetDeRecherche/Image/Foot_color_4.png",taillemin=4)